### Importing Packages

In [5]:
import pandas as pd 
import numpy as np 
import seaborn as sns 

import os
from loguru import logger

import warnings
warnings.filterwarnings('ignore')
import gc

### Common functions

In [72]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
#         print(f'{col}:{col_type}')
        
        if col_type not in [object,'category']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

### Importing Data

##### Importing train base data

In [7]:
for dirname, _, filenames in os.walk('/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/'):
    for filename in filenames:
        logger.info(f"Filename is:{filename}")

2024-02-28 16:29:51.199 | INFO     | __main__:<module>:3 - Filename is:train_credit_bureau_a_1_3.csv
2024-02-28 16:29:51.201 | INFO     | __main__:<module>:3 - Filename is:train_static_cb_0.csv
2024-02-28 16:29:51.203 | INFO     | __main__:<module>:3 - Filename is:train_applprev_1_0.csv
2024-02-28 16:29:51.204 | INFO     | __main__:<module>:3 - Filename is:train_person_2.csv
2024-02-28 16:29:51.205 | INFO     | __main__:<module>:3 - Filename is:train_base.csv
2024-02-28 16:29:51.206 | INFO     | __main__:<module>:3 - Filename is:train_tax_registry_a_1.csv
2024-02-28 16:29:51.207 | INFO     | __main__:<module>:3 - Filename is:train_static_0_0.csv
2024-02-28 16:29:51.208 | INFO     | __main__:<module>:3 - Filename is:train_credit_bureau_a_1_0.csv
2024-02-28 16:29:51.209 | INFO     | __main__:<module>:3 - Filename is:train_applprev_2.csv
2024-02-28 16:29:51.210 | INFO     | __main__:<module>:3 - Filename is:train_credit_bureau_a_2_6.csv
2024-02-28 16:29:51.211 | INFO     | __main__:<modul

## Base data

In [8]:
df_base = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_base.csv")

In [9]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526659 entries, 0 to 1526658
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   case_id        1526659 non-null  int64 
 1   date_decision  1526659 non-null  object
 2   MONTH          1526659 non-null  int64 
 3   WEEK_NUM       1526659 non-null  int64 
 4   target         1526659 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 58.2+ MB


In [10]:
logger.info(f"Number of unique records: {df_base['case_id'].nunique()}")

2024-02-28 16:29:52.608 | INFO     | __main__:<module>:1 - Number of unique records: 1526659


In [11]:
df_base = reduce_mem_usage(df_base)

Memory usage of dataframe is 58.24 MB
Memory usage after optimization is: 17.49 MB
Decreased by 70.0%


### Datasets with depth 0 

#### Static data

In [12]:
df_static_0 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_0_0.csv")
df_static_1 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_0_1.csv")

In [13]:
df_static_0 = reduce_mem_usage(df_static_0)
df_static_1 = reduce_mem_usage(df_static_1)

Memory usage of dataframe is 1279.85 MB
Memory usage after optimization is: 361.68 MB
Decreased by 71.7%
Memory usage of dataframe is 666.73 MB
Memory usage after optimization is: 190.29 MB
Decreased by 71.5%


In [14]:
logger.info(f"Number of unique records in 0: {df_static_0['case_id'].nunique()}")
logger.info(f"Number of unique records in 1: {df_static_1['case_id'].nunique()}")

2024-02-28 16:30:43.640 | INFO     | __main__:<module>:1 - Number of unique records in 0: 1003757
2024-02-28 16:30:43.654 | INFO     | __main__:<module>:2 - Number of unique records in 1: 522902


In [75]:
## Combining the datasets
df_static = pd.concat([df_static_0,df_static_1],ignore_index=True)

In [76]:
df_static = reduce_mem_usage(df_static)

Memory usage of dataframe is 768.74 MB
Memory usage after optimization is: 552.16 MB
Decreased by 28.2%


In [74]:
logger.info(f"Total records in static dataset: {df_static['case_id'].nunique()}")

2024-02-28 17:28:59.140 | INFO     | __main__:<module>:1 - Total records in static dataset: 1526659


#### Static CB

In [16]:
df_static_cb = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_cb_0.csv")

In [17]:
logger.info(f"Number of unique records in 0: {df_static_cb['case_id'].nunique()}")

2024-02-28 16:30:53.634 | INFO     | __main__:<module>:1 - Number of unique records in 0: 1500476


In [18]:
df_static_cb = reduce_mem_usage(df_static_cb)
df_static_cb.info()

Memory usage of dataframe is 606.73 MB
Memory usage after optimization is: 159.58 MB
Decreased by 73.7%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500476 entries, 0 to 1500475
Data columns (total 53 columns):
 #   Column                   Non-Null Count    Dtype   
---  ------                   --------------    -----   
 0   case_id                  1500476 non-null  int32   
 1   assignmentdate_238D      136996 non-null   category
 2   assignmentdate_4527235D  114978 non-null   category
 3   assignmentdate_4955616D  71633 non-null    category
 4   birthdate_574D           607871 non-null   category
 5   contractssum_5085716L    157329 non-null   float32 
 6   dateofbirth_337D         1385691 non-null  category
 7   dateofbirth_342D         36500 non-null    category
 8   days120_123L             1385691 non-null  float16 
 9   days180_256L             1385691 non-null  float16 
 10  days30_165L              1385691 non-null  float16 
 11  days360_512L             1385691 non-

In [77]:
gc.collect()

28670

##### Saving the new files 

In [78]:
df_base.to_parquet('train/train_base.parquet')
df_static.to_parquet('train/train_static.parquet')
df_static_cb.to_parquet('train/train_static_cb.parquet')

### Datasets with Depth 1